In [1]:
import tensorflow as tf
batch_size = 128
(x_train, Y_train), (x_test, Y_test) = tf.keras.datasets.cifar10.load_data()

In [2]:
import numpy as np
all_digits = np.concatenate([x_train, x_test])
all_output = np.concatenate([Y_train, Y_test])
all_output_one_hot = tf.one_hot(np.squeeze(all_output), 10)
all_digits = all_digits.astype("float32") / 255.0

2021-12-02 21:30:30.617050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-02 21:30:30.620060: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-02 21:30:30.620337: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-02 21:30:30.620757: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
all_digits = np.reshape(all_digits, (-1, 32, 32, 3))
dataset = tf.data.Dataset.from_tensor_slices((all_digits, all_output_one_hot))
dataset = dataset.shuffle(buffer_size=1024)
dataset = dataset.batch(batch_size).prefetch(1)

In [4]:
DATASET_SIZE=60000
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

full_dataset = dataset
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(test_size)
test_dataset = test_dataset.take(test_size)

In [32]:
input_block = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(224,224,3)),
        tf.keras.layers.Conv2D(64, 7, padding="same", strides=(2,2), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding="same"),
        tf.keras.layers.Conv2D(64, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(256, 1, padding="same", activation='relu'),
        
        tf.keras.layers.Conv2D(64, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(256, 1, padding="same", activation='relu'),
        
        tf.keras.layers.Conv2D(64, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(256, 1, padding="same", activation='relu')
    ], name = "input_block"
)

block_1 = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(56,56,256)),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding="same"),
        tf.keras.layers.Conv2D(128, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(128, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(512, 1, padding="same", activation='relu'),
        
        tf.keras.layers.Conv2D(128, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(128, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(512, 1, padding="same", activation='relu'),
        
        tf.keras.layers.Conv2D(128, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(128, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(512, 1, padding="same", activation='relu'),
        
        tf.keras.layers.Conv2D(128, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(128, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(512, 1, padding="same", activation='relu'),
    ], name = "block_1"
)


block_2 = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(28,28,512)),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding="same"),
        tf.keras.layers.Conv2D(256, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(256, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(1024, 1, padding="same", activation='relu'),
        
        tf.keras.layers.Conv2D(256, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(256, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(1024, 1, padding="same", activation='relu'),
        
        tf.keras.layers.Conv2D(256, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(256, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(1024, 1, padding="same", activation='relu'),
        
        tf.keras.layers.Conv2D(256, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(256, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(1024, 1, padding="same", activation='relu'),
        
        tf.keras.layers.Conv2D(256, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(256, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(1024, 1, padding="same", activation='relu'),
        
        tf.keras.layers.Conv2D(256, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(256, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(1024, 1, padding="same", activation='relu'),
    ], name = "block_2"
)

block_3 = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(14,14,1024)),
        tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding="same"),
        tf.keras.layers.Conv2D(512, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(512, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(2048, 1, padding="same", activation='relu'),
        
        tf.keras.layers.Conv2D(512, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(512, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(2048, 1, padding="same", activation='relu'),
        
        tf.keras.layers.Conv2D(512, 1, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(512, 3, padding="same", activation='relu'),
        tf.keras.layers.Conv2D(2048, 1, padding="same", activation='relu'),
        tf.keras.layers.AveragePooling2D(pool_size=(7,7), strides=(1, 1), padding='valid'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1000, activation="softmax")
    ], name = "block_3"
)

input_block.summary()
block_1.summary()
block_2.summary()
block_3.summary()

Model: "input_block"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_539 (Conv2D)         (None, 112, 112, 64)      9472      
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 56, 56, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_540 (Conv2D)         (None, 56, 56, 64)        4160      
                                                                 
 conv2d_541 (Conv2D)         (None, 56, 56, 64)        36928     
                                                                 
 conv2d_542 (Conv2D)         (None, 56, 56, 256)       16640     
                                                                 
 conv2d_543 (Conv2D)         (None, 56, 56, 64)        16448     
                                                       

In [37]:
class ForcedNet50(tf.keras.Model):
    def __init__(self, input_block, block_1, block_2, block_3):
        super(ForcedNet50, self).__init__()
        
        self.input_block = input_block
        self.output_input = tf.keras.Sequential(
            [
                tf.keras.layers.AveragePooling2D(pool_size=(56,56), strides=(1, 1), padding='valid'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(1000, activation="softmax")
            ]
        )
        
        self.block_1 = block_1
        self.output_1 = tf.keras.Sequential(
            [
                tf.keras.layers.AveragePooling2D(pool_size=(28,28), strides=(1, 1), padding='valid'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(1000, activation="softmax")
            ]
        )
        
        self.block_2 = block_2
        self.output_2 = tf.keras.Sequential(
            [
                tf.keras.layers.AveragePooling2D(pool_size=(14,14), strides=(1, 1), padding='valid'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(1000, activation="softmax")
            ]
        )
        
        self.block_3 = block_3
        #Total Params = 24549736
    def compile(self, optimizer, loss_fn):
        super(ForcedNet50, self).compile()
        self.optimizer = optimizer
        self.loss_fn = loss_fn
    
    def train_step(self, data):
        images = data[0]
        labels = data[1]
        
        with tf.GradientTape(persistent=True) as tape:
            x = self.input_block(images)
            predictions_input = self.output_input(x)
            
            x = self.block_1(x)
            predictions_1 = self.output_1(x)
            
            x = self.block_2(x)
            predictions_2 = self.output_2(x)

            predictions_3 = self.block_3(x)
            
            loss_input = self.loss_fn(labels, predictions_input)
            loss_1 = self.loss_fn(labels, predictions_1)
            loss_2 = self.loss_fn(labels, predictions_2)
            loss_3 = self.loss_fn(labels, predictions_3)
        
        grads_input = tape.gradient(loss_input, self.input_block.trainable_weights)
        grads_1 = tape.gradient(loss_1, self.block_1.trainable_weights)
        grads_2 = tape.gradient(loss_2, self.block_2.trainable_weights)
        grads_3 = tape.gradient(loss_3, self.block_3.trainable_weights)
        
        grads_input_output = tape.gradient(loss_input, self.output_input.trainable_weights)
        grads_1_output = tape.gradient(loss_1, self.output_1.trainable_weights)
        grads_2_output = tape.gradient(loss_2, self.output_2.trainable_weights)
        
        self.optimizer.apply_gradients(
            zip(grads_input, self.input_block.trainable_weights,)
        )
        self.optimizer.apply_gradients(
            zip(grads_input_output, self.input_output.trainable_weights)
        )
                                
        self.optimizer.apply_gradients(
            zip(grads_1, self.block_1.trainable_weights,)
        )
        self.optimizer.apply_gradients(
            zip(grads_1_output, self.output_1.trainable_weights)
        )
        
        self.optimizer.apply_gradients(
            zip(grads_2, self.block_2.trainable_weights)
        )
        
        self.optimizer.apply_gradients(
            zip(grads_2_output, self.output_2.trainable_weights)
        )
        
        self.optimizer.apply_gradients(
            zip(grads_3, self.block_3.trainable_weights)
        )
        
        return {"Input_Block_Loss": loss_input, "Block_1_Loss": loss_1,
                "Block_2_Loss": loss_2, "Block_3_Loss": loss_3}

In [38]:
ForcedLearner = ForcedNet50(input_block, block_1, block_2, block_3)
ForcedLearner.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=tf.keras.losses.CategoricalCrossentropy(),
)

In [39]:
model_resnet = tf.keras.applications.ResNet50(include_top=False, weights=None, input_shape=(224,224,3), classes=1000)
model_resnet.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))
model_resnet.summary()
#model_temp.fit(dataset, epochs=10000, callbacks=[callback])

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_63 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_63[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           